In [ ]:
%matplotlib notebook
import firedrake
import icepack, icepack.models, icepack.plot

In [ ]:
Lx, Ly = 50e3, 12e3
nx, ny = 16, 16

mesh = firedrake.RectangleMesh(nx, ny, Lx, Ly)

In [ ]:
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)
V = firedrake.VectorFunctionSpace(mesh, family='CG', degree=2)

In [ ]:
x, y = firedrake.SpatialCoordinate(mesh)

b_in, b_out = 200, -400
s_in, s_out = 850, 50

In [ ]:
from icepack.constants import (ice_density as ρ_I, water_density as ρ_W,
                               gravity as g)

h_in = s_in - b_in
δs_δx = (s_out - s_in) / Lx
τ_D = -ρ_I * g * h_in * δs_δx
print('{} kPa'.format(1000 * τ_D))

In [ ]:
u_in = 20, u_out = 2400
velocity_x = u_in + (u_out - u_in) * (x / Lx)**2

In [ ]:
T = 255.0

In [ ]:
from icepack.constants import weertman_sliding_law as m

C = (0.95 - 0.05 * x/Lx) * τ_D / u_in**(1/m)
# pressures, ramping factor

In [ ]:
import icepack.models.friction

def bed_friction_with_pressure(u, h, s, C):
    p_W = ρ_W * g * firedrake.max_value(0, h - s)
    p_I = ρ_I * g * h
    ϕ = 1 - p_W / p_I
    return icepack.models.friction.bed_friction(u, C * ϕ)